In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn; seaborn.set() #nice formatting


Balan 2009: http://adsabs.harvard.edu/abs/2009MNRAS.394.1936B
Exofit: http://www.star.ucl.ac.uk/~lahav/ExoFitv2.pdf

Equation for radial velocity:

$$
v(t_i) = K[ \sin(f_i + \omega) + e \sin(\omega)]
$$

$$
K = \frac{m_p}{m_s + m_p} \frac{2\pi}{T}\frac{a \sin i}{\sqrt{1 - e^2}}
$$

The true anomaly $f$ satisfies

$$
\cos(f_i) = \frac{\cos(E_i) - e}{1 - e\cos E_i}
$$

Rearranging this we can write
$$
f_i = 2 \arg\left(\sqrt{1 + e}\sin(E_i/2), \sqrt{1 - e} \cos(E_i/2)\right)
$$

The eccentric anomaly $E$ satisfies
$$
M = E - e\sin E
$$

and the mean anomaly is
$$
M = \frac{2\pi}{T}(t + \tau)
$$

and $\tau$ is the time of pericenter passage, which we'll parametrize with the parameter $\chi = \tau /  T$

Parameters in our model:

- $T$: orbital period
- $K$: amplitude of oscillation
- $V$: offset of oscillation
- $e$: eccentricity
- $\omega$: longitude of periastron
- $\chi$: dimensionless phase offset
- $s$: error term

In [ ]:
from scipy import optimize


@np.vectorize
def compute_E(M, e):
    """Solve Kepler's eqns for eccentric anomaly"""
    f = lambda E, M=M, e=e: E - e * np.sin(E) - M
    return optimize.brentq(f, 0, 2 * np.pi)


def radial_velocity(theta, t):
    """Compute radial velocity given orbital parameters"""
    T, K, V, e, omega, chi = theta[:6]
    
    # compute mean anomaly
    M = 2 * np.pi * ((t / T + chi) % 1)
    
    # solve for eccentric anomaly
    E = compute_E(M, e)
    
    # compute true anomaly
    f = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E / 2),
                       np.sqrt(1 - e) * np.cos(E / 2))
    
    # compute velocity
    return V - K * (np.sin(f + omega) + e * np.sin(omega))

Visualize this with IPython interact, to make sure the results make sense:

In [ ]:
from ipywidgets import interact

def plot_RV(T, K, V, e, omega, chi):
    t = np.linspace(0, 5, 200)
    theta = [T, K, V, e, omega, chi]
    plt.plot(t, radial_velocity(theta, t))
    
interact(plot_RV,
         T=(0, 5.), K=(0, 2000.), V=(-2000., 2000.),
         e=(0, 1.), omega=(0, 2 * np.pi), chi=(0, 1.));

In [ ]:
theta = [700, 60, 12, 0.38, 3.10, 0.67]
Nobs = 50

rng = np.random.RandomState(0)
t_sim = 1400 + 600 * rng.rand(Nobs)
err_sim = 1 + rng.rand(Nobs)
rv_sim = radial_velocity(theta, t_sim) + err_sim * rng.randn(Nobs)

plt.errorbar(t_sim, rv_sim, err_sim, fmt='.k');
xlim = plt.xlim()
t_fit = np.linspace(xlim[0], xlim[1], 500)
plt.plot(t_fit, radial_velocity(theta, t_fit), color='gray');